In [1]:
import evadb
cursor = evadb.connect(evadb_dir=("./eva_db")).cursor()

In [4]:
cursor.query("""
DROP TABLE IF EXISTS houses
""").df()
cursor.query("""
CREATE TABLE houses (
            area INTEGER,
            price INTEGER
            )"""
).df()
for area, price in zip([30, 35, 37, 40, 45, 50, 55, 60, 65, 70],
                       [300, 350, 370, 400, 450, 500, 550, 600, 650, 700]):
    cursor.query("""
    INSERT INTO houses (area, price) VALUES ({}, {})
    """.format(area, price)).df()
cursor.query("""
SELECT * FROM houses
""").df()


,_row_id,area,price
0,1,30,300
1,2,35,350
2,3,37,370
3,4,40,400
4,5,45,450
5,6,50,500
6,7,55,550
7,8,60,600
8,9,65,650
9,10,70,700


In [ ]:
# example
# CREATE OR REPLACE FUNCTION PredictHouseRent FROM
# ( SELECT number_of_rooms, number_of_bathrooms, days_on_market, rental_price FROM HomeRentals )
# TYPE Sklearn
# PREDICT 'rental_price';

cursor.query("""
CREATE OR REPLACE FUNCTION PredictHouseRent FROM
( SELECT area, price FROM houses )
TYPE Sklearn
PREDICT 'price';
""").df()


In [9]:
cursor.query("""
SELECT area, price, PredictHouseRent(area, price) FROM houses
""").df()

,area,price,price
0,30,300,325.370370
1,35,350,370.129630
2,37,370,370.129630
3,40,400,376.796296
4,45,450,417.222222
5,50,500,489.629630
6,55,550,539.814815
7,60,600,599.537037
8,65,650,634.722222
9,70,700,645.833333


In [19]:
from openai import OpenAI
import dotenv
dotenv.load_dotenv(dotenv.find_dotenv())
client = OpenAI()

style_prompt = "I need a course header image with lots of visual interest symbolically depicting and visualizing the field of study. Course name: "
course_prompt = "Deep Learning"
client.images.generate(
  model="dall-e-3",
  prompt=style_prompt + course_prompt,
  n=1,
  size="1024x1024",
)
# 7s    dalle-2 256x256
# 10.5s dalle-3

ImagesResponse(created=1700947238, data=[Image(b64_json=None, revised_prompt="Create an imaginative banner showcasing the field of Deep Learning. The header should encompass the essence of the subject in a visually engaging manner. Visual elements could include neural networks branching out with different layers represented, an abstract representation of a deep learning model architecture or algorithms at work, students interacting with complex data and technology, and various scientific symbols. The color scheme can be based on electric blues, deep purples, vibrant greens, or any other palette that gives a feeling of advanced technology. Embed the course name 'Deep Learning' prominently in the center of the image.", url='https://oaidalleapiprodscus.blob.core.windows.net/private/org-4K7HTKTulPjyLJIg8BfDLNHS/user-vPtmj6isZuiz3cJxP4a9MBZf/img-oEEKaEeSSRe5KsWB2MN9xHyC.png?st=2023-11-25T20%3A20%3A38Z&se=2023-11-25T22%3A20%3A38Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaad

In [27]:
# bind output of prev cell
out = Out[19]
out.data[0].url


'https://oaidalleapiprodscus.blob.core.windows.net/private/org-4K7HTKTulPjyLJIg8BfDLNHS/user-vPtmj6isZuiz3cJxP4a9MBZf/img-oEEKaEeSSRe5KsWB2MN9xHyC.png?st=2023-11-25T20%3A20%3A38Z&se=2023-11-25T22%3A20%3A38Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2023-11-25T12%3A31%3A28Z&ske=2023-11-26T12%3A31%3A28Z&sks=b&skv=2021-08-06&sig=4GA/XclV4ycau7pp8%2BtG4J5HoR6UF7B1BoxBaUhcGks%3D'